In [74]:
import os
from minio import Minio
import requests

In [75]:
client = Minio("minio-service.kubeflow.svc.cluster.local:9000",
    access_key="minio",
    secret_key="minio123",
    secure=False,           
)

In [76]:
type(client)

minio.api.Minio

In [77]:
bucket_name = "newtiledb"

In [78]:
def upload_files(bucket_name, file_location, client):
    found = False  # Initialize 'found' before the try block
    print("Current working directory:", os.getcwd())
    print("Listing directories in the current working directory:", os.listdir("."))
    print(f"Checking existence of {file_location}: ", os.path.exists(file_location))

    try:
        found = client.bucket_exists(bucket_name)
    except Exception as e:
        print("error trying to search for MinIO Bucket:", e)
        return  # Return early since we cannot proceed without knowing if the bucket exists

    if not found:
        try:
            client.make_bucket(bucket_name)
            print("Created bucket", bucket_name)
        except Exception as e:
            print("Failed to create bucket:", e)
            return  # Return early since we cannot proceed if the bucket cannot be created
    else:
        print("Bucket", bucket_name, "exists, we won't attempt to create one")
        
    # Ensure the directory exists
    if not os.path.isdir(file_location):
        print(f"The directory {file_location} does not exist.")
        return

    # Iterate through all files in the directory
    for file_name in os.listdir(file_location):
        # Construct the full file path
        source_file = os.path.join(file_location, file_name)
        # Check if it's a file and not a directory
        if os.path.isfile(source_file):
            try:
                # Upload the file
                client.fput_object(bucket_name, file_name, source_file)
                print(f"Successfully uploaded {file_name} to bucket {bucket_name}.")
            except Exception as e:
                print(f"Failed to upload {file_name}: {e}")


In [84]:
upload_files(bucket_name,"tiledb_demo/notebooks/documentation",client)

Current working directory: /home/jovyan
Listing directories in the current working directory: ['tmp_docs', '.gitconfig', '.mc', 'lost+found', '.local', '.conda', '.cache', '.bashrc', 'tiledb_demo', 'minio-binaries', '.ipynb_checkpoints', 'ezua-tutorials', 'tiledb_index', '.ipython', '.kube', '.jupyter', 'db_index.zip', '.bash_history']
Checking existence of tiledb_demo/notebooks/documentation:  True
Bucket newtiledb exists, we won't attempt to create one
Successfully uploaded tiledb_vector.txt to bucket newtiledb.
Successfully uploaded vector_database.txt to bucket newtiledb.
Successfully uploaded tiledb.txt to bucket newtiledb.
Successfully uploaded why_tile_blog.txt to bucket newtiledb.
Successfully uploaded RAG.txt to bucket newtiledb.
Successfully uploaded array_db.txt to bucket newtiledb.
Successfully uploaded human_in_loop.txt to bucket newtiledb.
Successfully uploaded embedding.txt to bucket newtiledb.
Successfully uploaded feature_store.txt to bucket newtiledb.
Successfully upl

In [107]:
data = {
  "instances": [{
    "input": "What makes tiledb a good vector store?",
    "num_docs": 4  # number of documents to retrieve
  }]
}

URL = "http://vectorstore-predictor.christensenc3526.svc.cluster.local/v1/models/vectorstore:predict"  # Adjust path as necessary

response = requests.post(URL, json=data, verify=False)  # 'verify=False' for self-signed certs
print(response)
#print(response.json())
print(response.text)

<Response [200]>
{"predictions":["In summary, TileDB's architecture and features make it an excellent choice for managing vector data, especially in scenarios requiring scalability, performance, and flexibility in data management. Its ability to handle various data types, combined with efficient storage, querying capabilities, and cloud-native support, positions it as a versatile and powerful vector store solution.","TileDB is an array database, and its main strength is that it can morph into practically any data modality and application, delivering unprecedented performance and alleviating the data infrastructure in an organization. A vector is simply a 1D array, therefore, TileDB is the most natural database choice for delivering amazing vector search functionality.","TileDB is designed as a universal data management solution, which means it can handle a wide variety of data types, including tabular, sparse, and dense multi-dimensional array data. When considering TileDB as a vector 

In [113]:
URL = URL = "http://llm-transformer.christensenc3526.svc.cluster.local/v1/models/llm:predict"

In [114]:
data = {
  "instances": [{
      "system": "You are an AI assistant. You will be given a task. You must generate a detailed answer.",
      "instruction": "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.",
      "input": "what part of tiledbs architecture makes it a good choice for a vectorstore?",
      "max_tokens": 5000,
      "top_k": 100,
      "top_p": 0.4,
      "num_docs": 3,
      "temperature": 0.2
  }]
}
response = requests.post(URL, json=data,verify=False)

In [115]:
print(response)
#print(response.json())
print(response.text)

<Response [200]>
{"predictions":["TileDB's architecture and features make it an excellent choice for managing vector data due to its ability to handle various data types, efficient storage, querying capabilities, and cloud-native support. TileDB is designed as a universal data management solution that can handle tabular, sparse, and dense multi-dimensional array data, making it the most natural database choice for delivering amazing vector search functionality. Additionally, TileDB's ability to morph into practically any data modality and application makes it a versatile and powerful vector store solution. Therefore, TileDB is an excellent choice for managing vector data in scenarios requiring scalability, performance, and flexibility in data management."]}
